# Cassandra

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cql

In [13]:
%%cql
DROP KEYSPACE black;

'No results.'

In [14]:
%%cql
CREATE KEYSPACE black 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [15]:
%cql USE black;

'No results.'

## Creacción de las tablas

En **Cassandra** vamos a crear 2 tablas principales, una donde almacenamos los movimientos en formato desnormalizado, y otra que nos servirá para almacenar los importes acumulados por cliente.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

En la tabla que almacena los movimientos, como no tenemos una PK clara, vamos a utilizar un tipo UUID (marca temporal única) donde guardaremos un TIMESTAMP con la hora de creación del registro

<br><br> 

<img src="images/Modelo%20Cassandra.png" width="800" height="500">

<br><br> 

In [16]:
%%cql 
CREATE TABLE acum_movimientos_nombre (
    nombre          text,
    importe         counter,
    PRIMARY KEY (nombre)
)

'No results.'

In [17]:
%%cql 
CREATE TABLE movimientos (
    uid             uuid,
    fecha           date,
    hora            int,
    minuto          int,
    importe         decimal,
    comercio        text,
    actividad_completa text,
    actividad       text,
    nombre          text,
    funcion         text,
    organizacion    text,
    PRIMARY KEY ((nombre), importe, fecha, hora, minuto)
)
WITH CLUSTERING ORDER BY (importe DESC);

'No results.'

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos

In [18]:
%%cql
CREATE MATERIALIZED VIEW vm_movimientos_by_actividad AS
   SELECT * FROM movimientos
   WHERE actividad IS NOT NULL 
         and importe IS NOT NULL 
         and fecha IS NOT NULL
         and hora IS NOT NULL 
         and minuto IS NOT NULL 
         and nombre IS NOT NULL
   PRIMARY KEY (actividad, importe, fecha,hora,minuto,nombre)
   WITH CLUSTERING ORDER BY (importe desc)

'No results.'

## Carga de datos en pandas

Cargamos los datos en formato Excel y los desnormalizamos ...

In [19]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6823 non-null   object        
 6   actividad_completa  7623 non-null   object        
 7   actividad           7623 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6003 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


### Carga de datos en Cassandra

In [20]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('black')

A destacar varios aspectos:
- El tipo de las fechas. Hay que dejarlo con el tipo **Date** de Python para no tener problemas (El tipo de la columna es también Date)
- Los nulos en Pandas son del tipo NumPy.nan, por lo hay convertirlos a None para que la inserción en Cassandra sea correcta
- En la tabla de importes acumulados guardamos el importe en un dato de tipo **counter**, que tiene un tipo entero, por lo que multiplicamos el dato por 100 para poder almacenarlo correctamete

In [21]:
df.loc[df.comercio.isnull(),['comercio']] = None
df.loc[df.organizacion.isnull(),['organizacion']] = None
df.loc[df.actividad.isnull(),['actividad']] = None
df.loc[df.actividad_completa.isnull(),['actividad_completa']] = None

In [22]:
import dateutil

def insert_movientos(df):
    
    sql_insert = """
INSERT INTO movimientos (
uid,
importe,
actividad,
fecha,
actividad_completa,
comercio,
funcion,
hora,
minuto,
organizacion,
nombre
) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

    for index in df.index:
        fecha = df['fecha'][index]
        
        data = [
            df["importe"][index],
            df["actividad"][index],
            fecha.date(),
            df["actividad_completa"][index],
            df["comercio"][index],
            df["funcion"][index],
            df["hora"][index],
            df["minuto"][index],
            df["organizacion"][index],
            df["nombre"][index],
        ]
        
        session.execute(sql_insert, data)
        
        importe_int = int(round(df["importe"][index] * 100))        
        session.execute("UPDATE acum_movimientos_nombre SET importe = importe + %s WHERE nombre = %s", 
                        [importe_int, 
                         df["nombre"][index]]
        )

Ya podemos insertar los datos ...

In [23]:
insert_movientos(df)

## Consulta de datos

Función de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [24]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### Los 10 movimientos mas caros

Para resolver esta query con la base de datos tendríamos que tener un PARTITION KEY única para todos los registros

In [25]:
sql = """
SELECT nombre, fecha, actividad_completa, importe
FROM MOVIMIENTOS
"""
df = execute_query(sql)

In [26]:
df.sort_values('importe', ascending = False).head(10)

,nombre,fecha,actividad_completa,importe
4699,Ildefonso José Sánchez Barcoj,2009-12-30,EL CORTE INGLES,16921.75
4409,Miguel Blesa de la Parra,2006-04-04,EL CORTE INGLES,12597.26
397,Ramón Ferraz Ricarte,2003-04-03,AGENCIAS DE VIAJES,5998.35
398,Ramón Ferraz Ricarte,2009-08-23,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",5909.29
1433,Estanislao Rodríguez-Ponga Salamanca,2010-12-20,HIPERCOR SUPERMERCADOS EL CORTE INGLES,4985.0
5732,Matías Amat Roca,2006-08-02,AGENCIAS DE VIAJES,4972.0
3371,María Carmen Cafranga Cavestany,2010-10-07,AGENCIAS DE VIAJES,4336.0
4700,Ildefonso José Sánchez Barcoj,2010-03-25,EL CORTE INGLES,4320.5
2698,Carmen Contreras Gómez,2008-12-19,JOYERIAS Y RELOJERIAS,4218.0
2699,Carmen Contreras Gómez,2007-02-11,AIR EUROPA,4175.87


### Los movimientos de una persona concreta

Se resuelve la query con la base de datos, ya que la PARTITION KEY es el campo **nombre**, y los registros están ordenados por importe

In [27]:
sql = """
select nombre, fecha, actividad_completa, importe
from movimientos
where nombre = 'Javier de Miguel Sánchez'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad_completa,importe
0,Javier de Miguel Sánchez,2003-02-11,EL CORTE INGLES,1190.0
1,Javier de Miguel Sánchez,2004-05-18,MELIA (CADENA DE HOTELES),593.39
2,Javier de Miguel Sánchez,2008-06-12,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",572.0
3,Javier de Miguel Sánchez,2004-04-06,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",538.97
4,Javier de Miguel Sánchez,2009-05-30,CONFECCION TEXTIL EN GENERAL,350.0
5,Javier de Miguel Sánchez,2009-12-22,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",280.0
6,Javier de Miguel Sánchez,2008-04-24,SERV.ORG.PUBL.Y GRANDES EMPRESAS,269.92
7,Javier de Miguel Sánchez,2010-01-19,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",260.72
8,Javier de Miguel Sánchez,2009-01-28,"HOSPITALES,SANATORIOS Y CONSULTAS MEDIC",250.0
9,Javier de Miguel Sánchez,2005-09-19,CONFECCION TEXTIL EN GENERAL,222.5


### Las 10 personas que mas han gastado

Esta consulta se pueder resolver mediante el acumulado que habíamos creado ad-doc. Observa que el importe es entero, por lo que hay que dividirlo por 100 para convertirlo a decimal

In [28]:
sql = """
select *
from acum_movimientos_nombre
"""
df = execute_query(sql)

In [29]:
df.assign(importe = df.importe / 100).sort_values('importe', ascending = False).head(10)

,nombre,importe
57,Ildefonso José Sánchez Barcoj,71331.33
55,Miguel Blesa de la Parra,60325.29
73,Ricardo Morado Iglesias,46435.74
72,José Antonio Moral Santín,45026.71
4,Ramón Ferraz Ricarte,44438.94
67,Matías Amat Roca,42663.76
30,Mariano Pérez Claver,32121.01
54,Francisco Baquero Noriega,31558.70
34,Carmen Contreras Gómez,30159.02
46,Antonio Romero Lázaro,27954.38


### Los 10 movimientos mas caros por actividad

Esta consulta se pueder resolver mediante la vista materializa que hemos creado ad-hoc para resolver esta consulta

In [30]:
sql = """
select nombre, fecha, actividad, actividad_completa, importe
from vm_movimientos_by_actividad
where actividad = 'HOGAR'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad,actividad_completa,importe
0,Jesús Pedroche Nieto,2009-07-05,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",3512.0
1,Jesús Pedroche Nieto,2011-10-20,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",3061.0
2,Francisco Baquero Noriega,2008-05-15,HOGAR,FLORES Y PLANTAS,2764.19
3,Mariano Pérez Claver,2006-06-30,HOGAR,IKEA,1217.94
4,Jesús Pedroche Nieto,2009-01-02,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1000.41
5,Jesús Pedroche Nieto,2011-08-22,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",945.82
6,Maria Mercedes de la Merced Monge,2006-09-12,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",843.25
7,Ildefonso José Sánchez Barcoj,2011-04-02,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",760.0
8,Alejandro Couceiro Ojeda,2006-07-04,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",620.0
9,Enrique de la Torre Martínez,2007-04-11,HOGAR,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",575.9
